In [1]:
from torch.utils.data import DataLoader
from dataset import AarizDataset
path = "Aariz Dataset"
dataset = AarizDataset( path ,"TRAIN")

batch_size = 5
training_data = AarizDataset( path ,"TRAIN")


# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle=True)

In [2]:
import torch
from torch import nn
from torchvision import models

class CephNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.ResNet = models.resnet50(pretrained=True)
        # ResNet Output (b, 2048)
        num_features = self.ResNet.fc.in_features
        # Number of flattened features (b, 58)
        self.ResNet = nn.Linear (num_features, 58)

    def forward(self, x):
        # (b,58)
        flat_output = self.ResNet(x)
        # (1, 29, 2)
        output = flat_output.view(-1,29,2)

        return output

CephNet = CephNet()


print(CephNet)

C:\Users\samco\PycharmProjects\Cephanator\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\samco\PycharmProjects\Cephanator\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CephNet(
  (ResNet): Linear(in_features=2048, out_features=58, bias=True)
)


In [3]:
from helper import  draw_batch

for (images, landmarks,_) in train_dataloader:
    print(f"Shape of Image Batch: {images.shape}")
    print(f"Shape of Landmark Batch: {landmarks.shape} {landmarks.dtype}")
    # draw_batch(images, landmarks)
    output = CephNet.forward(images)
    print(f"Shape of Output batch: {output.shape}")
    break



Shape of Image Batch: torch.Size([5, 3, 2750, 2200])
Shape of Landmark Batch: torch.Size([5, 29, 2]) torch.float32


RuntimeError: mat1 and mat2 must have the same dtype, but got Byte and Float